In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pmdarima as pmd
from statsmodels.tsa.statespace.dynamic_factor import DynamicFactor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [2]:
# Load the dataset
df = pd.read_csv("./BTC.csv")

# Drop the unnecessary columns.
df.drop(['Open', 'High', 'Low', 'Change %'], axis=1, inplace=True)

# Convert the "Date" column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y')
                                    
# Sort the DataFrame by date
df = df.sort_values('Date')

# Set Date index
df.set_index('Date', inplace=True)

df['Vol.'] = df['Vol.'].str.replace('K', 'e3')
df['Vol.'] = df['Vol.'].str.replace('M', 'e6')
df['Vol.'] = pd.to_numeric(df['Vol.'], errors='coerce')
df

,Price,Vol.
Date,,
2017-12-01,10861.5,131920.0
2017-12-02,10912.7,86830.0
2017-12-03,11246.2,122130.0
2017-12-04,11623.9,93170.0
2017-12-05,11667.1,89690.0
...,...,...
2023-03-28,27262.2,94160.0
2023-03-29,28350.4,109320.0
2023-03-30,28029.5,122510.0


### 2. Split the data into training, test and validate sets

In [3]:
train_size = int(0.7 * len(df))
test_size = int(0.15 * len(df))
val_size = len(df) - train_size - test_size

train_data = df[:train_size]
test_data = df[train_size:train_size+test_size]
val_data = df[train_size+test_size:]

print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)
print("Validate shape:", val_data.shape)

Train shape: (1363, 2)
Test shape: (292, 2)
Validate shape: (293, 2)


In [4]:
train_size = int(0.6 * len(df))
test_size = int(0.2 * len(df))
val_size = len(df) - train_size - test_size

train_data = df[:train_size]
test_data = df[train_size:train_size+test_size]
val_data = df[train_size+test_size:]

print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)
print("Validate shape:", val_data.shape)

Train shape: (1168, 2)
Test shape: (389, 2)
Validate shape: (391, 2)


In [5]:
train_size = int(0.7 * len(df))
test_size = int(0.2 * len(df))
val_size = len(df) - train_size - test_size

train_data = df[:train_size]
test_data = df[train_size:train_size+test_size]
val_data = df[train_size+test_size:]

print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)
print("Validate shape:", val_data.shape)

Train shape: (1363, 2)
Test shape: (389, 2)
Validate shape: (196, 2)


## 3. Perform Dynamic Factor Model

### 3.1 Find factor_order from p order of ARIMA Model and perform Dynamic Factor Model

In [6]:
features = ['Price']

arima_model = pmd.auto_arima(train_data[features], start_p=1, start_q=1,
                         test='adf',
                         max_p=3, max_q=3, m=12,
                         start_P=0, seasonal=False,
                         d=None, D=0, trace=True,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)

factor_order = arima_model.order[0]

Performing stepwise search to minimize aic


C:\Users\PHUTRAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\pmdarima\arima\_validation.py:62: UserWarning: m (12) set for non-seasonal fit. Setting to 0
  warnings.warn("m (%i) set for non-seasonal fit. Setting to 0" % m)


 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=22395.474, Time=0.30 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=22399.786, Time=0.04 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=22396.362, Time=0.07 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=22397.037, Time=0.08 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=22399.016, Time=0.02 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=22391.747, Time=0.28 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=22392.217, Time=0.10 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=22393.134, Time=0.42 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=22393.161, Time=0.56 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=22391.175, Time=0.34 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=22391.702, Time=0.12 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=22393.227, Time=0.55 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=22392.038, Time=0.16 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=22394.669, Time=0.84 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=22389.776, Time=0.21

In [7]:
df_model = DynamicFactor(endog=train_data, k_factors=1, factor_order=factor_order, enforce_stationarity=False)
df_model_fit = df_model.fit(disp=False)
df_model_fit.summary()

C:\Users\PHUTRAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\PHUTRAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Dep. Variable:,"['Price', 'Vol.']",No. Observations:,1363
Model:,"DynamicFactor(factors=1, order=2)",Log Likelihood,-41356.254
Date:,"Thu, 15 Jun 2023",AIC,82724.508
Time:,18:32:30,BIC,82755.813
Sample:,12-01-2017,HQIC,82736.226
,- 08-24-2021,,
Covariance Type:,opg,,
Ljung-Box (L1) (Q):,"1171.51, 980.17",Jarque-Bera (JB):,"2689.60, 13546.56"
Prob(Q):,"0.00, 0.00",Prob(JB):,"0.00, 0.00"
Heteroskedasticity (H):,"4.09, 3.07",Skew:,"2.08, 2.95"
Prob(H) (two-sided):,"0.00, 0.00",Kurtosis:,"8.48, 17.28"


In [24]:
def evaluate_dynamic_factor_model(data):
    predictions = df_model_fit.predict(start=data.index[0], end=data.index[len(data)-1])
    data = data[features]
    predictions = predictions[features]
    mse = mean_squared_error(data, predictions)
    mae = mean_absolute_error(data, predictions)
    mape = mean_absolute_percentage_error(data, predictions)
    rmse = mean_squared_error(data, predictions, squared=False)
    return predictions, mse, mae, mape, rmse
predictions = df_model_fit.predict(start=train_data.index[0], end=train_data.index[len(train_data)-1])
print(predictions)

                   Price          Vol.
2017-12-01      0.000000  0.000000e+00
2017-12-02   2562.279644 -2.829383e+05
2017-12-03   3896.160347 -4.302313e+05
2017-12-04   4061.502912 -4.484891e+05
2017-12-05   4106.902885 -4.535024e+05
...                  ...           ...
2021-08-20  18183.620416 -2.007916e+06
2021-08-21  18861.108423 -2.082727e+06
2021-08-22  19692.659503 -2.174551e+06
2021-08-23  19770.861618 -2.183186e+06
2021-08-24  19790.407857 -2.185344e+06

[1363 rows x 2 columns]


### 3.2 Evaluate on Validate data

In [21]:
y_pred_val, val_mse, val_mae, val_mape, val_rmse = evaluate_dynamic_factor_model(train_data)

print("Price predict on validate data:", val_rmse)

Price predict on validate data: 15445.22712483379
